In [1]:
import os
import shutil

import numpy as np
import yaml

In [11]:
# run this to reset

src_dir = 'demo'
dest_dir = 'images/new'

files = os.listdir(src_dir)
shutil.copytree(src_dir, dest_dir, dirs_exist_ok=True)

'images/new'

In [2]:
new_images = "images/new"  # stores new data
labeled_images = "images/labeled"  # stores data that has labels
unlabeled_images = "images/unlabeled"  # stores data that doesnt have labels
bad_images = "images/bad"  # stores images the model cant understand / bad quality

In [15]:
# filter new images

# find labeled images
cache = {}
for image in os.listdir(new_images):
    name = image.split(".")[0]
    if name in cache:
        cache[name] += 1
    else:
        cache[name] = 1

# move files to correct folders
for image in os.listdir(new_images):
    name = image.split(".")[0]
    value = cache[name]
    old_path = os.path.join(new_images, image)
    if value == 1: # unlabeled
        new_path = os.path.join(unlabeled_images, image)
        # os.rename(old_path, new_path)
    elif value == 2: # labeled
        if image.endswith(".txt"):
            new_path = os.path.join(labeled_images, "labels", image)
            os.rename(old_path, new_path)
        elif image.endswith(".jpg"):
            new_path = os.path.join(labeled_images, "images", image)
            os.rename(old_path, new_path)
    else:
        # something went wrong
        print(image)

In [3]:
# train model with labeled images
import yolov5.train as train
opt = train.parse_opt(True)

# change some values
opt.__setattr__("data", "yolov5/data/dataset.yaml")
opt.__setattr__("batch_size", 8)
opt.__setattr__("img", 640)
opt.__setattr__("epochs", 3)
opt.__setattr__("noval", True)  # validate only last epoch
opt.__setattr__("noplots", True)  # dont save plots
opt.__setattr__("name", "erik_test")
opt.__setattr__("weights", "")
# opt.__setattr__("cfg", "yolov5n6.yaml")  # use untrained model
opt.__setattr__("weights", "yolov5/yolov5s.pt")  # use trained model

train.main(opt)

train: weights=yolov5/yolov5s.pt, cfg=, data=yolov5/data/dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=True, noautoanchor=False, noplots=True, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=erik_test, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, img=640
YOLOv5 🚀 2022-11-25 Python-3.8.15 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
requirements: /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v2/yolov5/requirements.txt not found, check failed.


Model summary: 214 layers, 7235389 parameters, 7235389 gradients, 16.6 GFLOPs

Transferred 349/349 items from yolov5/yolov5s.pt
AMP: checks passed ✅
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 60 weight(decay=0.0005), 60 bias
train: Scanning /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v2/images/labeled/labels.cache... 126 images, 0 backgrounds, 0 corrupt: 100%|██████████| 126/126 00:00
val: Scanning /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v2/images/labeled/labels.cache... 126 images, 0 backgrounds, 0 corrupt: 100%|██████████| 126/126 00:00

AutoAnchor: 4.28 anchors/target, 0.995 Best Possible Recall (BPR). Current anchors are a good fit to dataset ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to yolov5/runs/train/erik_test9
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
        0/2      1.73G    0.04531    0.06819    0.02078         84        6

In [57]:
# test model
import torch
import yaml
import pandas as pd
from pathlib import Path
from PIL import Image

# load model
path_to_weights = 'yolov5/runs/train/erik_test6/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=path_to_weights)  # custom model
device = torch.device('cuda')
model = model.to(device)

# use labeled images to test model
# it is normal if confidence scores and iou are high
# if image has low score then:
#  model hasn't learned that image
#  image is wrong
#
# use this step to figure out the best threshold for human intervention

conf_thres=0.30,  # confidence threshold
iou_thres=0.45,  # NMS IOU threshold

# load yaml file
names = yaml.safe_load(Path("yolov5/data/dataset.yaml").read_text())["names"]

# Images
images_path = os.path.join(labeled_images, "images")
annotation_path = os.path.join(labeled_images, "labels")
n = 0
for im in os.listdir(images_path):

    # annotation data
    ano_file = os.path.join(annotation_path, f"{im.split('.')[0]}.txt")
    ano_data = pd.read_csv(ano_file, sep=" ", names=["class", "x", "y", "w", "h"])
    ano_data = ano_data.sort_values(by=['class'],ignore_index=True)

    # image file
    image = Image.open(os.path.join(images_path, im))
    result = model(image)
    image_data = result.pandas().xywhn[0]
    image_data = image_data.sort_values(by=['class'],ignore_index=True)

    # compare if has same annotations
    if not image_data["class"].equals(ano_data["class"]):
        print(image_data)
        print(ano_data)

        # todo filter wrong




Using cache found in /home/erik/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-25 Python-3.8.15 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

Fusing layers... 
Model summary: 157 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


requirements: /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v2/yolov5/requirements.txt not found, check failed.
    xcenter   ycenter     width    height  confidence  class     name
0  0.535206  0.548458  0.883403  0.889043    0.676413     23  giraffe
   class         x         y         w         h
0     23  0.658478  0.592133  0.677002  0.779766
1     23  0.391581  0.556305  0.546862  0.887391
    xcenter   ycenter     width    height  confidence  class        name
0  0.481495  0.447091  0.885659  0.769133    0.303044     59         bed
1  0.167410  0.956806  0.334821  0.084235    0.296948     59         bed
2  0.860523  0.069351  0.067873  0.058502    0.522216     77  teddy bear
   class         x         y         w         h
0     59  0.510930  0.442073  0.978141  0.872188
1     77  0.858305  0.073521  0.074922  0.059833
    xcenter   ycenter     width    height  confidence  class    name
0  0.514052  0.484199  0.971895  0.968399    0.920599      0  person
1  0.432459  0.9

In [ ]:
# predict unlabeled data
# if conf is low then human intervention is necessary

# Images unlabeled
images = []
for im in os.listdir(unlabeled_images):
    if im.endswith(".jpg"):
        images.append(Image.open(os.path.join(unlabeled_images, im)))
# Inference
result = model(images)
print(result.pandas())